In [ ]:
!pip install -q diffusers transformers accelerate torch
!pip install -q flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video

# 1. Load the 2B model in FP16 to save memory
# CogVideoX-2B is the smaller, Colab-friendly version of the 5B model
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-2b",
    torch_dtype=torch.float16
)

# 2. Critical Optimizations for Free Tier
pipe.enable_model_cpu_offload()      # Moves parts of the model to CPU when not active
pipe.enable_sequential_cpu_offload() # Extreme memory saving
pipe.vae.enable_slicing()            # Decodes video frames one by one to prevent VRAM spikes
pipe.vae.enable_tiling()             # Handles high-resolution tiles separately

# 3. Generate Video
prompt = "A hyper-realistic cinematic shot of a futuristic city at night, heavy rain, neon lights reflecting on puddles, 4k, highly detailed."

video_frames = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50, # 50 is standard for quality; 30 for speed
    num_frames=49,          # CogVideoX uses (8n + 1) frames. 49 frames = ~6 seconds
    guidance_scale=6.0,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

# 4. Export as MP4
export_to_video(video_frames, "output_video.mp4", fps=8)

In [ ]:
prompt = "An aerial cinematic wide shot looking down through thick clouds at a sprawling mega-city. Massive skyscrapers pierce through the fog, glowing with golden amber lights. Flying vehicles leave long light trails like glowing veins between the towers. Moody, dark atmosphere, photorealistic, intricate textures, IMAX film aesthetic.."

video_frames = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50, # 50 is standard for quality; 30 for speed
    num_frames=49,          # CogVideoX uses (8n + 1) frames. 49 frames = ~6 seconds
    guidance_scale=6.0,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

# 4. Export as MP4
export_to_video(video_frames, "output_video.mp4", fps=8)